In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
def remove_txt_headers(path):
    """Removes lines from txt file that were added by LabChart as headers. Writes new file.

    :param path: Path to txt file
    :type path: str
    """
    
    no_lines = ['Interval', 'ExcelDateTime', 'TimeFormat', 'DateFormat', 'ChannelTitle', 'Range', 'UnitName', 'TopValue', 
                'BottomValue']  # Lines to remove
    
    # Read raw lines
    with open(f'raw_labchart/{path}', 'r') as f:
        lines = f.readlines()
    
    # Remove lines
    lines_arr = np.array(lines)
    lines_arr = lines_arr[[not any(x in y for x in no_lines) for y in lines_arr]]

    # Write new file
    s = ''.join(lines_arr)
    with open('headers_removed/{0}_out.txt'.format(path.split('.')[0]), 'w') as f:
        f.write(s)

In [30]:
# Import all trials data
dfs = [pd.read_excel('ap_data.xlsx', f'Sheet{x}') for x in range(1, 5)]
arrs = [x.to_numpy() for x in dfs]

# Average all trials into one DataFrame
df = pd.DataFrame(np.mean(arrs, axis=0), columns = pd.read_excel('ap_data.xlsx').columns)

In [4]:
# Averaged dose response curves
def plot_dose_reponse(df, y, title):
    """Plots dose reponse given the signal (y).

    :param df: Response data
    :type df: pandas DataFrame
    :param y: Name of signal column in df
    :type y: str
    :param title: Title for plot image
    :type title: str
    :return: Plot
    :rtype: plotly figure
    """
    fig = px.line(df, x='ml_amount', y=y, color='voltage', width=1200, height=600)
    
    # Format plot
    fig.update_layout(
        title=dict(text=title, x=0.5, font_size=40),
        xaxis=dict(title_text='Phenol Amount (mL)', title_font_size=30, tickfont_size=20),
        yaxis=dict(title_text='Average Response', title_font_size=30, tickfont_size=20),
        legend=dict(title_text='Stimulation Voltage (V)', font_size=25)
    )
    fig.update_traces(
        line=dict(width=7, dash='dash')
    )
    fig.update_yaxes(range=[-0.25, 2.25])  # Set y-axis max and min for consistency
    
    return fig

# Response to phenol
phenol_fig = plot_dose_reponse(df, 'ap', 'Action Potential in Response to Phenol Amount')
phenol_fig.show()

# Control reponse
control_fig = plot_dose_reponse(df, 'control', 'Action Potential in Control')
control_fig.show()

In [5]:
def plot_ap_sig(df, title, rolling=None, ap_xs=[]):
    fig = make_subplots(rows=2, cols=1, row_heights=[0.7, 0.3], x_title='Time (s)', y_title='Voltage (μV)',
                        shared_xaxes=True, shared_yaxes=True)
    
    # Add traces to subplots
    fig.add_trace(
        go.Scatter(x=df['time'], y=(df.rolling(rolling).mean()['signal'] if rolling else df['signal']), name='Recording',
                   mode='lines'),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=df['time'], y=df['stimulus'], name='Stimulus', mode='lines'),
        row=2, col=1
    )
    for x in ap_xs:
        fig.add_vrect(
            x0=x - 0.0005, x1=x + 0.0005,
            fillcolor="LightSalmon", opacity=0.5, layer="below", line_width=0,
            row=1, col=1
        )
    
    # Format plot
    fig.update_traces(
        line_width=4
    )
    fig.update_layout(
        width=800,
        height=400,
        title=dict(text=title, x=0.5, font_size=35),
        legend=dict(title='Signal', font_size=20)
    )
    fig.update_xaxes(tickfont_size=16)
    fig.update_yaxes(tickfont_size=16)
    
    # Change axes label sizes
    for x in fig.layout.annotations:
        x["font"] = {'size': 25}
    
    return fig

# AP data directory
ap_dir = 'select_ap_signals'

# Plot no AP case
no_ap = pd.read_excel(f'{ap_dir}/trial11.xlsx')
no_ap['signal'] = -no_ap['signal']  # Flip polarity in post
no_ap_fig = plot_ap_sig(no_ap, 'No Action Potential')
no_ap_fig.show()

# Plot medial and lateral AP case
two_ap = pd.read_excel(f'{ap_dir}/trial210.xlsx')
two_ap_fig = plot_ap_sig(two_ap, 'Medial and Lateral Action Potential', rolling=5, ap_xs=[0.006675, 0.0124])
two_ap_fig.show()

# Plot medial only AP case
one_ap = pd.read_excel(f'{ap_dir}/trial266.xlsx')
one_ap_fig = plot_ap_sig(one_ap, 'Medial Only Action Potential', ap_xs=[0.0135])
one_ap_fig.show()